In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Загружаем модель и токенизатор, кладем их на GPU
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
texts = [
    "Как привлечь к себе благосостояние?",
    "Че делать когда мучает совесть после пьянки?",
    "Почему вы хотите знать , что будет завтра?",
]

In [6]:
tokenizer.apply_chat_template([{"role": "user", "content": texts[0]}], return_tensors="pt")

tensor([[128000, 128006,    882, 128007,    271, 106523, 108349, 113493,   4929,
           7820, 104001, 117934,  23630,  25262,  61390, 101486,     30, 128009,
         128006,  78191, 128007,    271]])

In [21]:
# Подавать просто так входы в модель нельзя. Различные instruct-LLM обучены на различных форматах 
# и нужно обязательно ими пользоваться. Если этого не делать, то качество генераций может как снизиться, так 
# и стать совсем бессмысленным
print(tokenizer.apply_chat_template([{"role": "system", "content": "Дай ответ в стиле mail.ru ответов"}, {"role": "user", "content": texts[0]}], tokenize=False))

# Вот так выглядит форматирование для llama3.1
"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Как привлечь к себе благосостояние?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Дай ответ в стиле mail.ru ответов<|eot_id|><|start_header_id|>user<|end_header_id|>

Как привлечь к себе благосостояние?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




'\n<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nКак привлечь к себе благосостояние?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n'

In [22]:
for text_idx, text in  enumerate(texts):
    print("\n\n")
    print(f"-----Текст {text_idx}-----")
    print(text)
    input_ids = tokenizer.apply_chat_template(
        [{"role": "system", "content": "Дай ответ в стиле mail.ru ответов"}, 
         {"role": "user", "content": text}], 
        return_tensors="pt")  # токенизируем текст, возаращаем тензоры для pytorch
    input_ids = input_ids.to(device) # кладем тензоры на то же устройство (gpu, cpu), что и модель
    # генерируем токены с сэмплингом и температурой, о том, как это работает, можно узнать на курсе!
    # также некоторые технические аргументы тоже опустим, но если очень хочется, то обо всех можно почитать в документации
    # https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig
    for i in range(1):
        outputs = model.generate(
            input_ids=input_ids,
            do_sample=True,
            temperature=0.8,
            max_length=200,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )
        # проводим детокенизацию, т.е. превращаем сгенерированные токены обратно в текст
        generated_token_ids = outputs[0][input_ids.size(1):]
        candidate = tokenizer.decode(generated_token_ids)
        print(f"\n-----Кандидат {i}-----")
        print(candidate)




-----Текст 0-----
Как привлечь к себе благосостояние?

-----Кандидат 0-----
Чтобы привлечь к себе благосостояние, необходимо поменять свои мысли и настроения. Приносите благодарность за то, что у вас есть, а не жалуитесь на то, что вам не хватает. Также важно научиться ценить и любить себя, а не критиковать и унижать. Это поможет привлечь положительные энергии в вашу жизнь.<|eot_id|>



-----Текст 1-----
Че делать когда мучает совесть после пьянки?

-----Кандидат 0-----
Если после пьянки мучает совесть, есть несколько шагов, которые можно предпринять, чтобы решить эту проблему:

1. **Принеси в порядок свое жилище**: После выпитого алкоголя может остаться много грязи и беспорядка. Уборка дома поможет вам отвлечься от негативных мыслей и почувствовать себя лучше.

2. **Выпей воды**: Пьянка часто приводит к дефициту воды в организме. Пейте много воды, чтобы разбавить алкоголь и снять симптомы похмелья.

3. **Немного поспите**: Если возможно, позвольте себе отдохнуть. Похмелье часто быв